# Dance of the bees
Batchrunner Data

Door Roeland Oostdam?, Thijme de Bruijn, Gerrit van de Bunt

ABD Challenge

# Gebruiksaanwijzing;
1. **Geef het absolute pad op naar de folder waar de folder 'IntelligentBeesChallenge' zich in bevindt.**
2. Stel parameters op voor de simulatie, deze kan je onderverdelen in variable en fixed parameters die je hier ook onder kan definieren. **Let op: voor variable parameters moet je een range object opgeven**
3. Zorg ervoor dat alle parameters zijn ingedeeld in of variable of fixed parameters en run dan alle cells.

# Parameters:
"height" : Hoogte van de grid van de omgeving \
"width : Breedte van de grid van de omgeving \
"init_bees" : Aantal bijen in de omgeving aan het begin \
"init_flowers" : Aantal bloemvelden in de omgeving aan het begin \
"min_nectar" : Minimale hoeveelheid nectar beschikbaar in bloemvelden \
"max_nectar" : Maximale hoeveelheid nectar beschikbaar in bloemvelden \
"init_min_nectar_grade" : Minimale kwaliteit nectar wat kan voorkomen \
"init_max_nectar_grade" : Maximale kwaliteit nectar wat kan voorkome \
"nectar_respawn_interval" : Aantal timesteps dat minimaal verstrijkt voordat nectar terug komt op een bloemenveld \
"max_bee_energy" : Capaciteit energie die een bij kan hebben nadat hij bij een bijenkorf geweest is en aan het begin \
"collect_negative_value_nectar" : Collect negative value nectar (?) \
"perception_range" : Hoe ver de bij kan zien om bloemenvelden/nectar te identificeren en op te slaan in memory.

In [8]:
# BATCHRUNNER PARAMS

# Voorbeeld:
# Als dit het pad is waar de notebook zich in bevindt;
# C:\Users\gvand\Projecten\Pycharm\ABD\IntelligentBeesChallenge\Dance of the bees Batchrunner Data.ipynb
# Geef dan dit op als project path:
# proj_path = r"C:\Users\gvand\Projecten\Pycharm\ABD"

proj_path = r"C:\Users\gvand\Projecten\Pycharm\ABD"

fixed_params = {
    "height": 10,
    "width": 10,
    "init_bees": 1,
    "init_flowers": 6,
    "init_min_nectar_grade": 1,
    "init_max_nectar_grade": 30,
    "min_nectar": 1,
    "max_nectar": 1,
    "nectar_respawn_interval": 50,
    "collect_negative_value_nectar": True,
    "perception_range": 1,
}

variable_params = {
    "max_bee_energy": range(30, 50, 10)
}

In [9]:
# Zet content root op om import problemen te voorkomen
import sys
sys.path.insert(1, proj_path)

In [10]:
# Importeer benodigde onderdelen voor de batchrunner
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from IntelligentBeesChallenge.bee_simulation.model import BeeSimulation, get_nectar_per_t, get_bee_energy, get_hive_energy, \
    get_nectar_collected

In [11]:
batch_run = BatchRunner(BeeSimulation,
                        variable_parameters=variable_params,
                        fixed_parameters=fixed_params,
                        iterations=3,
                        model_reporters={
                            "Nectar/T": get_nectar_per_t,
                            "Bee energy": get_bee_energy,
                            "Nectar stored": get_hive_energy,
                            "Nectar Collected": get_nectar_collected,
                        })
batch_run.run_all()
model_data = batch_run.get_model_vars_dataframe()

1it [00:00,  6.14it/s]

Current State: explore
[[  -22.   -20.   -18.   -16.   -18.   -20.   -22.   -24.   -26.   -28.]
 [  -20.   -18.   -16.   -14.   -16.   -18.   -20.   -22.   -24.   -26.]
 [  -18.   -16.   -14.   -12.   -14.   -16.   -18.   -20.   -22.   -24.]
 [  -16.   -14.   -12.   -10.   -12.   -14.   -16.   -18.   -20.   -22.]
 [  -14.   -12.   -10.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [  -12.   -10.    -8.    -6.    -8.   -10.   -12.   -14.   -16.   -18.]
 [  -10.    -8.    -6.    -4.    -6.    -8.   -10.   -12.   -14.   -16.]
 [   -8.    -6.    -4. -1000.    -4.    -6.    -8.   -10.   -12.   -14.]
 [   -6.    -4. -1000. -1000. -1000.    -4.    -6.    -8.   -10.   -12.]
 [   -8.    -6.    -4. -1000.    -4.    -6.    -8.   -10.   -12.   -14.]]
Current State: explore
[[  -21.   -19.   -17.   -17.   -19.   -21.   -23.   -25.   -27.   -29.]
 [  -19.   -17.   -15.   -15.   -17.   -19.   -21.   -23.   -25.   -27.]
 [  -17.   -15.   -13.   -13.   -15.   -17.   -19.   -21.   -23.   -25.]
 [  

2it [00:00,  5.72it/s]


[[-1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.   -18.]
 [-1000. -1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.]
 [-1000. -1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.]
 [-1000. -1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.]
 [-1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.   -18.]
 [  -10.   -10.   -10.   -10.   -10.   -12.   -14.   -16.   -18.   -20.]
 [  -12.   -12.   -12.   -12.   -12.   -14.   -16.   -18.   -20.   -22.]
 [  -14.   -14.   -14.   -14.   -14.   -16.   -18.   -20.   -22.   -24.]
 [  -16.   -16.   -16.   -16.   -16.   -18.   -20.   -22.   -24.   -26.]
 [  -18.   -18.   -18.   -18.   -18.   -20.   -22.   -24.   -26.   -28.]]
Current State: explore
[[-1000. -1000. -1000. -1000. -1000.   -11.   -13.   -15.   -17.   -19.]
 [-1000. -1000. -1000. -1000. -1000. -1000.   -11.   -13.   -15.   -17.]
 [-1000. -1000. -1000. -1000. -1000. -1000.   -11.   -13.   -15.   -17.]
 [-1000. -1000. -1000. -10

Current State: explore
[[-1000. -1000. -1000. -1000. -1000.    -9.   -11.   -13.   -15.   -17.]
 [-1000. -1000. -1000. -1000. -1000.    -9.   -11.   -13.   -15.   -17.]
 [-1000. -1000. -1000. -1000. -1000.    -9.   -11.   -13.   -15.   -17.]
 [-1000. -1000. -1000. -1000. -1000.    -9.   -11.   -13.   -15.   -17.]
 [-1000. -1000. -1000. -1000. -1000.    -9.   -11.   -13.   -15.   -17.]
 [  -11.    -9. -1000. -1000.    -9.   -11.   -13.   -15.   -17.   -19.]
 [  -13.   -11.    -9.    -9.   -11.   -13.   -15.   -17.   -19.   -21.]
 [  -15.   -13.   -11.   -11.   -13.   -15.   -17.   -19.   -21.   -23.]
 [  -17.   -15.   -13.   -13.   -15.   -17.   -19.   -21.   -23.   -25.]
 [  -19.   -17.   -15.   -15.   -17.   -19.   -21.   -23.   -25.   -27.]]
Current State: explore
[[-1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.   -18.]
 [-1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.   -18.]
 [-1000. -1000. -1000. -1000. -1000.   -10.   -12.   -14.   -16.   -18.]
 [-1

4it [00:00,  6.13it/s]


Current State: explore
[[  -16.   -14.   -12.   -10.    -8.    -8.    -8.    -8.   -10.   -12.]
 [-1000. -1000. -1000. -1000. -1000. -1000. -1000. -1000.    -8.   -10.]
 [-1000. -1000. -1000. -1000. -1000. -1000. -1000. -1000. -1000.    -8.]
 [-1000. -1000. -1000. -1000. -1000. -1000. -1000. -1000.    -6.    -8.]
 [-1000. -1000. -1000. -1000. -1000.    -6.    -6.    -6.    -8.   -10.]
 [  -16.   -14.   -12.   -10.    -8.    -8.    -8.    -8.   -10.   -12.]
 [  -18.   -16.   -14.   -12.   -10.   -10.   -10.   -10.   -12.   -14.]
 [  -20.   -18.   -16.   -14.   -12.   -12.   -12.   -12.   -14.   -16.]
 [  -22.   -20.   -18.   -16.   -14.   -14.   -14.   -14.   -16.   -18.]
 [  -24.   -22.   -20.   -18.   -16.   -16.   -16.   -16.   -18.   -20.]]
Current State: explore
[[  -17.   -15.   -13.   -11.    -9.    -7.    -7.    -7.    -9.   -11.]
 [-1000. -1000. -1000. -1000. -1000. -1000. -1000. -1000.    -7.    -9.]
 [-1000. -1000. -1000. -1000. -1000. -1000. -1000. -1000. -1000.    -7.]
 [-

6it [00:00,  7.07it/s]

Current State: return_to_hive
Current State: return_to_hive
Current State: return_to_hive
Current State: return_to_hive
Current State: return_to_hive
move_to_target origin same as target
Current State: return_to_hive
move_to_target origin same as target
Current State: explore
[[  -24.   -22.   -20.   -18.   -16.   -14.   -16.   -18.   -20.   -22.]
 [  -22.   -20.   -18.   -16.   -14.   -12.   -14.   -16.   -18.   -20.]
 [  -20.   -18.   -16.   -14.   -12.   -10.   -12.   -14.   -16.   -18.]
 [  -18.   -16.   -14.   -12.   -10.    -8.   -10.   -12.   -14.   -16.]
 [  -16.   -14.   -12.   -10.    -8.    -6.    -8.   -10.   -12.   -14.]
 [  -14.   -12.   -10.    -8.    -6.    -4.    -6.    -8.   -10.   -12.]
 [  -12.   -10.    -8.    -6.    -4. -1000.    -4.    -6.    -8.   -10.]
 [  -10.    -8.    -6.    -4. -1000. -1000. -1000.    -4.    -6.    -8.]
 [  -12.   -10.    -8.    -6.    -4. -1000.    -4.    -6.    -8.   -10.]
 [  -14.   -12.   -10.    -8.    -6.    -4.    -6.    -8.   -10.  

In [12]:
model_data

,max_bee_energy,Run,Bee energy,Nectar Collected,Nectar stored,Nectar/T,height,width,init_bees,init_flowers,init_min_nectar_grade,init_max_nectar_grade,min_nectar,max_nectar,nectar_respawn_interval,collect_negative_value_nectar,perception_range
0,30,0,0,56,0,0.528302,10,10,1,6,1,30,1,1,50,True,1
1,30,1,0,93,0,0.650350,10,10,1,6,1,30,1,1,50,True,1
2,30,2,0,7,0,0.122807,10,10,1,6,1,30,1,1,50,True,1
3,40,3,0,140,0,0.700000,10,10,1,6,1,30,1,1,50,True,1
4,40,4,0,22,0,0.268293,10,10,1,6,1,30,1,1,50,True,1
5,40,5,0,2,0,0.032258,10,10,1,6,1,30,1,1,50,True,1
